In [29]:
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
from html.parser import HTMLParser
import numpy as np
import pandas as pd

In [30]:
url = 'https://www.hockey-reference.com/leagues/NHL_2019_standings.html'
soup = BeautifulSoup(urlopen(url), "html.parser")

In [31]:
# Find the proper table using the caption tag
for caption in soup.find_all('caption'):
    if caption.get_text() == 'Expanded Standings Table':
        table = caption.find_parent('table', {"id":"standings"})

In [32]:
#Table data into one large array
all_teams = []
#Grab all the data
for row in table.find_all('tr'):
    for cell in row.find_all('td'):
        all_teams.append(cell.text)

In [33]:
#divide the all_teams data into their own arrays
def teamsplitter(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [34]:
#21 columns in chart
all_teams = list(teamsplitter(all_teams,21))
#Data->numpy
all_teams = np.array(all_teams)
#numpy-> Pandas DataFrame
teamtable = pd.DataFrame(all_teams)
# print(teamtable)
#clean data & rename headers
teamtable = teamtable.drop([4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],axis=1)
teamtable = teamtable.rename(columns={0:"team",1:"Overall",2:"Shootout",3:"Overtime"})

teamtable.head()

,team,Overall,Shootout,Overtime
0,Tampa Bay Lightning,62-16-4,6-1,7-3
1,Calgary Flames,50-25-7,0-4,5-3
2,Boston Bruins,49-24-9,2-3,9-6
3,Washington Capitals,48-26-8,4-1,5-7
4,New York Islanders,48-27-7,5-5,6-2


In [8]:
# Striping out the strings of the records

teamname = teamtable.drop(["Overall","Shootout","Overtime"],axis=1)
Overall_math = teamtable.Overall.str.split("-",expand=True).rename(columns={0:"wins",1:"loss",2:"OT_L"}).astype(int)
Shootout_math = teamtable.Shootout.str.split("-",expand=True).drop([1],axis=1).rename(columns={0:"SOWins"}).astype(int)
Overtime_math = teamtable.Overtime.str.split("-",expand=True).drop([1],axis=1).rename(columns={0:"OTWins"}).astype(int)

In [40]:
#Join math tables
pts_math = teamname.join(Overall_math)
pts_math = pts_math.join(Shootout_math)
pts_math = pts_math.join(Overtime_math)

pts_math.head()

,team,wins,loss,OT_L,SOWins,OTWins
0,Tampa Bay Lightning,62,16,4,6,7
1,Calgary Flames,50,25,7,0,5
2,Boston Bruins,49,24,9,2,9
3,Washington Capitals,48,26,8,4,5
4,New York Islanders,48,27,7,5,6


In [41]:
# Math to get all the pts totals...(Keeping to show the work)
pts_math['OT_W'] = pts_math.SOWins + pts_math.OTWins
pts_math['true_Wins'] = pts_math.wins - pts_math.OT_W
pts_math['True_Wins_Pts'] = pts_math.true_Wins*3
pts_math['OT_W_Pts'] = pts_math.OT_W*2
pts_math['OT_L_Pts'] = pts_math.OT_L*1

pts_math.head()

,team,wins,loss,OT_L,SOWins,OTWins,OT_W,true_Wins,True_Wins_Pts,OT_W_Pts,OT_L_Pts
0,Tampa Bay Lightning,62,16,4,6,7,13,49,147,26,4
1,Calgary Flames,50,25,7,0,5,5,45,135,10,7
2,Boston Bruins,49,24,9,2,9,11,38,114,22,9
3,Washington Capitals,48,26,8,4,5,9,39,117,18,8
4,New York Islanders,48,27,7,5,6,11,37,111,22,7


In [42]:
# THE NEW TOTAL POINTS!!
pts_math['new_Record']= pts_math.true_Wins.astype(str).str.cat([pts_math.OT_W.astype(str),pts_math.OT_L.astype(str),Overall_math.loss.astype(str)],sep='-')
pts_math['PTS_Total']= pts_math.True_Wins_Pts+pts_math.OT_W_Pts+pts_math.OT_L_Pts

pts_math.head()

,team,wins,loss,OT_L,SOWins,OTWins,OT_W,true_Wins,True_Wins_Pts,OT_W_Pts,OT_L_Pts,new_Record,PTS_Total
0,Tampa Bay Lightning,62,16,4,6,7,13,49,147,26,4,49-13-4-16,177
1,Calgary Flames,50,25,7,0,5,5,45,135,10,7,45-5-7-25,152
2,Boston Bruins,49,24,9,2,9,11,38,114,22,9,38-11-9-24,145
3,Washington Capitals,48,26,8,4,5,9,39,117,18,8,39-9-8-26,143
4,New York Islanders,48,27,7,5,6,11,37,111,22,7,37-11-7-27,140


In [43]:
pts_math['current_Points']=(pts_math.wins*2)+(pts_math.OT_L)
# clean for export
pts_math = pts_math.drop(['OT_W', 'True_Wins_Pts', 'OT_W_Pts', 'OT_L_Pts'], axis=1)
pts_math = pts_math.rename(columns={"OT_L":"overtime_Loss","SOWins":"shoot_Out_Wins","OTWins":"overtime_Wins","PTS_Total":"points"})

In [44]:
pts_math['current_Rank'] = pts_math['current_Points'].rank(ascending=False)

pts_math['new_Rank'] = pts_math['points'].rank(ascending=False)
pts_math = pts_math.sort_values(by=["points"],ascending=False)

pts_math.head()

,team,wins,loss,overtime_Loss,shoot_Out_Wins,overtime_Wins,true_Wins,new_Record,points,current_Points,current_Rank,new_Rank
0,Tampa Bay Lightning,62,16,4,6,7,49,49-13-4-16,177,128,1.0,1.0
1,Calgary Flames,50,25,7,0,5,45,45-5-7-25,152,107,2.5,2.0
2,Boston Bruins,49,24,9,2,9,38,38-11-9-24,145,107,2.5,3.0
3,Washington Capitals,48,26,8,4,5,39,39-9-8-26,143,104,4.0,4.0
4,New York Islanders,48,27,7,5,6,37,37-11-7-27,140,103,5.0,5.5


In [52]:
team_props = pd.read_json('teams.json',typ='frame')

finish_table = pts_math.join(team_props,on='team')

In [55]:
finish_table = pts_math.join(team_props,on='team')
finish_table = finish_table.set_index('abb')

finish_table.head()

,team,wins,loss,overtime_Loss,shoot_Out_Wins,overtime_Wins,true_Wins,new_Record,points,current_Points,current_Rank,new_Rank,conference,division
abb,,,,,,,,,,,,,,
TB,Tampa Bay Lightning,62,16,4,6,7,49,49-13-4-16,177,128,1.0,1.0,Eastern,Atlantic
CAL,Calgary Flames,50,25,7,0,5,45,45-5-7-25,152,107,2.5,2.0,Western,Pacific
BOS,Boston Bruins,49,24,9,2,9,38,38-11-9-24,145,107,2.5,3.0,Eastern,Atlantic
WAS,Washington Capitals,48,26,8,4,5,39,39-9-8-26,143,104,4.0,4.0,Eastern,Metropolitan
NYI,New York Islanders,48,27,7,5,6,37,37-11-7-27,140,103,5.0,5.5,Eastern,Metropolitan
